
# Group Sparse inverse covariance for multi-subject connectome

This example shows how to estimate a connectome on a group of subjects
using the group sparse inverse covariance estimate.

.. include:: ../../../examples/masker_note.rst


In [18]:
# Libraries and Paths
import os
import sys
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn import datasets, plotting
from nilearn.maskers import NiftiLabelsMasker
from nilearn.connectome import GroupSparseCovarianceCV
import logging

# Paths and parameters
curr_dir = f'/user_data/csimmon2/git_repos/ptoc'
sys.path.insert(0, curr_dir)
import ptoc_params as params

raw_dir = params.raw_dir
results_dir = '/user_data/csimmon2/git_repos/ptoc/results'

# Load subject information
sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
subjects_to_skip = []
subs = sub_info[(sub_info['group'] == 'control') & (~sub_info['sub'].isin(subjects_to_skip))]['sub'].tolist()


In [14]:
import numpy as np

from nilearn import plotting

n_subjects = 19  # subjects to consider for group-sparse covariance (max: 40)


def plot_matrices(cov, prec, title, labels):
    """Plot covariance and precision matrices, for a given processing."""
    prec = prec.copy()  # avoid side effects

    # Put zeros on the diagonal, for graph clarity.
    size = prec.shape[0]
    prec[list(range(size)), list(range(size))] = 0
    span = max(abs(prec.min()), abs(prec.max()))

    # Display covariance matrix
    plotting.plot_matrix(
        cov,
        cmap=plotting.cm.bwr,
        vmin=-1,
        vmax=1,
        title=f"{title} / covariance",
        labels=labels,
    )
    # Display precision matrix
    plotting.plot_matrix(
        prec,
        cmap=plotting.cm.bwr,
        vmin=-span,
        vmax=span,
        title=f"{title} / precision",
        labels=labels,
    )

## Fetching datasets



In [21]:
from nilearn import datasets

# Instead of MSDL atlas, use Schaefer
atlas = datasets.fetch_atlas_schaefer_2018(n_rois=200, yeo_networks=7, resolution_mm=2)

# Create our dataset structure
func_filenames = []
for sub in subs:  # using subs from your sub_info loading
    subject_runs = [
        f'{raw_dir}/{sub}/ses-01/derivatives/reg_standard/filtered_func_run-0{run}_standard.nii.gz'
        for run in [1, 2, 3]
    ]
    func_filenames.extend(subject_runs)
    
# print basic information on the dataset
print(f"First subject functional nifti image (4D) is at: {func_filenames[0]}")

# Print all subjects and their runs
#for i, filename in enumerate(func_filenames):
#    print(f"Subject functional nifti image {i} (4D) is at: {filename}")


First subject functional nifti image (4D) is at: /lab_data/behrmannlab/vlad/hemispace/sub-025/ses-01/derivatives/reg_standard/filtered_func_run-01_standard.nii.gz


## Extracting region signals



In [ ]:
from nilearn.maskers import NiftiMapsMasker

masker = NiftiMapsMasker(
    atlas.maps,
    resampling_target="maps",
    detrend=True,
    high_variance_confounds=True,
    low_pass=None,
    high_pass=0.01,
    t_r=2,
    standardize="zscore_sample",
    standardize_confounds="zscore_sample",
    memory="nilearn_cache",
    memory_level=1,
    verbose=2,
)
masker.fit()

subject_time_series = []
# If you have confound files, add their paths here similar to func_filenames
# confound_filenames = [f'{raw_dir}/{subject}/...' for subject in subjects]

# If you have confounds:
# for func_filename, confound_filename in zip(func_filenames, confound_filenames):
#     print(f"Processing file {func_filename}")
#     region_ts = masker.transform(func_filename, confounds=confound_filename)
#     subject_time_series.append(region_ts)

# If you don't have confounds:
for func_filename in func_filenames:
    print(f"Processing file {func_filename}")
    region_ts = masker.transform(func_filename)
    subject_time_series.append(region_ts)

[MemorizedFunc(func=<function high_variance_confounds at 0x7f818599c8b0>, location=nilearn_cache/joblib)]: Clearing function cache identified by nilearn/image/image/high_variance_confounds


[NiftiMapsMasker.fit] loading regions from None
Processing file /lab_data/behrmannlab/vlad/hemispace/sub-025/ses-01/derivatives/reg_standard/filtered_func_run-01_standard.nii.gz
________________________________________________________________________________
[Memory] Calling nilearn.image.image.high_variance_confounds...
high_variance_confounds('/lab_data/behrmannlab/vlad/hemispace/sub-025/ses-01/derivatives/reg_standard/filtered_func_run-01_standard.nii.gz')
_________________________________________high_variance_confounds - 10.4s, 0.2min
________________________________________________________________________________
[Memory] Calling nilearn.maskers.base_masker._filter_and_extract...
_filter_and_extract('/lab_data/behrmannlab/vlad/hemispace/sub-025/ses-01/derivatives/reg_standard/filtered_func_run-01_standard.nii.gz', 
<nilearn.maskers.nifti_maps_masker._ExtractionFunctor object at 0x7f818193de20>, { 'allow_overlap': True,
  'clean_kwargs': {},
  'detrend': True,
  'dtype': None,
  'h

## Computing group-sparse precision matrices



In [ ]:
from nilearn.connectome import GroupSparseCovarianceCV

gsc = GroupSparseCovarianceCV(verbose=2)
gsc.fit(subject_time_series)


from sklearn.covariance import GraphicalLassoCV

gl = GraphicalLassoCV(verbose=2)
gl.fit(np.concatenate(subject_time_series))

## Displaying results



In [ ]:
atlas_img = atlas.maps
atlas_region_coords = plotting.find_probabilistic_atlas_cut_coords(atlas_img)
labels = atlas.labels

plotting.plot_connectome(
    gl.covariance_,
    atlas_region_coords,
    edge_threshold="90%",
    title="Covariance",
    display_mode="lzr",
)
plotting.plot_connectome(
    -gl.precision_,
    atlas_region_coords,
    edge_threshold="90%",
    title="Sparse inverse covariance (GraphicalLasso)",
    display_mode="lzr",
    edge_vmax=0.5,
    edge_vmin=-0.5,
)
plot_matrices(gl.covariance_, gl.precision_, "GraphicalLasso", labels)

title = "GroupSparseCovariance"
plotting.plot_connectome(
    -gsc.precisions_[..., 0],
    atlas_region_coords,
    edge_threshold="90%",
    title=title,
    display_mode="lzr",
    edge_vmax=0.5,
    edge_vmin=-0.5,
)
plot_matrices(gsc.covariances_[..., 0], gsc.precisions_[..., 0], title, labels)

plotting.show()